In [1]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Matsuo Lab/AIエンジニアリング2025/lecture-ai-engineering/day3/homework

/content/drive/MyDrive/Matsuo Lab/AIエンジニアリング2025/lecture-ai-engineering/day3/homework


In [3]:
!pip install -r requirements.txt

## ライブラリのインポートとAPIキー設定

In [4]:
# ライブラリのインポート
from __future__ import annotations

import os
import json
from pathlib import Path
from typing import List, Dict, Tuple, Any
import datetime

import faiss
import numpy as np
import pdfplumber
from sentence_transformers import SentenceTransformer
import google.generativeai as genai

# APIキー設定
# Google Colabの場合:
from google.colab import userdata
key = userdata.get("GOOGLE_API_KEY")
if key:
    os.environ["GOOGLE_API_KEY"] = key
else:
    print("Google Colab Secretsに GOOGLE_API_KEY を設定してください。")

if not os.getenv("GOOGLE_API_KEY"):
    raise RuntimeError(
        "GOOGLE_API_KEY が見つかりません。"
        "適切な方法 (Colab Secrets, 環境変数, .env, または直接コード内) で設定してください。"
    )

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
print("APIキーの設定が完了しました。")

APIキーの設定が完了しました。


## パラメータ設定

In [5]:
# --- 実験パラメータ ---
PDF_PATH_PARAM = Path("令和6年4月1日施行.pdf")
QUESTIONS_PATH_PARAM = Path("questions.json") # 外部質問ファイル
EMBED_MODEL_NAME_PARAM = "intfloat/multilingual-e5-large"
GEMINI_MODEL_NAME_PARAM = "gemini-2.5-pro-exp-03-25"
CHUNK_SIZE_PARAM = 500
CHUNK_OVERLAP_PARAM = 200
TOP_K_PARAM = 5
TEMPERATURE_PARAM = 0.1
MAX_OUTPUT_TOKENS_PARAM = 1024
OUTPUT_DIR_PARAM = Path("results_notebook") # Notebook用の出力ディレクトリ
RUN_ID_PARAM = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_notebook")

# デフォルト質問データ (questions.json がない場合に使用)
DEFAULT_QUESTIONS_DATA_PARAM: List[Dict[str, str]] = [
    {
        "id": "q_default_1",
        "text": "発明の新規性喪失の例外（特許法30条）の適用を受けるには、証明書を特許出願の日から何日以内に提出する必要がありますか？",
        "answer": "30日以内",
        "notes": "LLM単体でも答えられる可能性が高い基本的な知識問題。"
    },
    {
        "id": "q_default_2",
        "text": "仮専用実施権は、どのような場合に移転が認められますか？具体的に列挙してください。",
        "answer": "発明の実施の事業とともにする場合、特許を受ける権利を有する者の承諾を得た場合、相続その他の一般承継の場合",
        "notes": "複数の条件を正確に列挙する必要がある問題。"
    },
    {
        "id": "q_default_3",
        "text": "特許法第65条第6項に規定する補償金請求権の消滅時効は何年ですか？また、その起算点はいつですか？",
        "answer": "原則として、特許出願人が発明の実施の事実及び実施をした者を知った時から3年です。ただし、これらの事実を特許権設定登録前に知った場合は、特許権設定登録の日から3年となります。",
        "notes": "LLMが単体では正確に答えられない可能性のある、特定の条文の細かい内容（時効期間と起算点）を問う問題。"
    },
    {
        "id": "q_default_4",
        "text": "特許権の存続期間延長登録の出願ができる場合のうち、「その特許発明の実施に法律の規定による許可その他の処分であつて当該処分の目的、手続等からみて当該特許権の効力の制限の程度その他の事情を勘案して政令で定めるものを受けることが必要であつたために、その特許発明の実施をすることができない期間があつたとき」に該当しない場合はどのような場合ですか？",
        "answer": "具体的には、以下のような場合が該当しません。（１）そもそも法令に基づく許可等が必要ない場合、（２）法令に基づく許可等が必要であっても、それが政令で定める許可等に該当しない場合、（３）特許発明の実施ができなかった期間が、当該許可等を得るために必要であったことに起因するものではない場合。",
        "notes": "複雑な条件文を理解し、除外規定を特定する必要がある。LLM単体では困難な可能性。"
    },
    {
        "id": "q_default_5",
        "text": "令和6年4月1日施行の改正特許法において、意匠権の存続期間満了後のいわゆる「グレースピリオド」に関する新たな規定は導入されましたか？その内容を簡潔に説明してください。",
        "answer": "いいえ、令和6年4月1日施行の改正特許法には、意匠権の存続期間満了後のグレースピリオドに関する直接的な新規定は含まれていません。意匠法には新規性喪失の例外規定（意匠法第4条）が存在しますが、これは存続期間満了後ではなく出願前の行為に関するものです。",
        "notes": "最新の法改正内容と、特定の制度（意匠のグレースピリオド）の有無を問う。参照文書に直接的な記述がない場合、LLMは推測で答えるか、不明と答える可能性がある。RAGで関連情報から判断させる。"
    }
]


# パラメータを辞書としてまとめておく (スクリプトのARGSの代わり)
class ArgsNamespace: # argparse.Namespaceの簡易的な模倣
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

ARGS_PARAM = ArgsNamespace(
    pdf_path=PDF_PATH_PARAM,
    questions_path=QUESTIONS_PATH_PARAM,
    embed_model=EMBED_MODEL_NAME_PARAM,
    gemini_model=GEMINI_MODEL_NAME_PARAM,
    chunk_size=CHUNK_SIZE_PARAM,
    chunk_overlap=CHUNK_OVERLAP_PARAM,
    top_k=TOP_K_PARAM,
    temperature=TEMPERATURE_PARAM,
    max_output_tokens=MAX_OUTPUT_TOKENS_PARAM,
    output_dir=OUTPUT_DIR_PARAM,
    run_id=RUN_ID_PARAM,
)

print("パラメータ設定完了:")
for arg, value in vars(ARGS_PARAM).items():
    print(f"  {arg}: {value}")

パラメータ設定完了:
  pdf_path: 令和6年4月1日施行.pdf
  questions_path: questions.json
  embed_model: intfloat/multilingual-e5-large
  gemini_model: gemini-2.5-pro-exp-03-25
  chunk_size: 500
  chunk_overlap: 200
  top_k: 5
  temperature: 0.1
  max_output_tokens: 1024
  output_dir: results_notebook
  run_id: 20250512_070831_notebook


## ユーティリティ関数定義 (テキスト抽出、チャンキング)

In [6]:
# --- ユーティリティ関数定義 ---

def extract_text(pdf_path: Path) -> str:
    """Concatenate all page texts from a PDF."""
    if not pdf_path.exists():
        # ARGS_PARAM はセル2で定義されている想定
        raise FileNotFoundError(
            f"PDFファイルが見つかりません: {pdf_path}. "
            f"セル2の PDF_PATH_PARAM を確認するか、デフォルトの場所に配置してください。"
        )
    texts: List[str] = []
    print(f"PDFファイルを開いています: {pdf_path}")
    with pdfplumber.open(pdf_path) as pdf:
        print(f"合計ページ数: {len(pdf.pages)}")
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text(x_tolerance=1, y_tolerance=1) or ""
            texts.append(page_text)
            if (i + 1) % 10 == 0 or (i + 1) == len(pdf.pages): # 10ページごと、または最後のページで進捗を表示
                 print(f"  {i+1}ページ目までのテキストを抽出完了...")
    print("PDFからのテキスト抽出が完了しました。")
    return "\n".join(texts)

def chunk_text(text: str, size: int, overlap: int) -> List[str]:
    """Simple fixed‑length overlapping chunk splitter (by characters)."""
    chunks: List[str] = []
    start = 0
    text_length = len(text)
    if text_length == 0:
        return []

    print(f"テキストをチャンク分割中... (全長: {text_length}, サイズ: {size}, オーバーラップ: {overlap})")
    while start < text_length:
        end = start + size
        chunks.append(text[start:end])
        if end >= text_length:
            break
        start += size - overlap
        if start >= text_length: # オーバーラップにより最終チャンクの開始点がテキスト長を超える場合
            break

    if not chunks and text: # テキストがチャンクサイズより小さい場合
        chunks.append(text)
    print(f"チャンク分割完了。生成されたチャンク数: {len(chunks)}")
    return chunks

print("ユーティリティ関数 (extract_text, chunk_text) が定義されました。")

ユーティリティ関数 (extract_text, chunk_text) が定義されました。


## RAGコンポーネント グローバル変数定義

In [7]:
# --- RAGコンポーネント グローバル変数 ---
# これらは initialize_rag_components 関数で設定されます。

CHUNKS_GLOBAL: List[str] = []
EMBEDDER_GLOBAL: SentenceTransformer | None = None
INDEX_GLOBAL: faiss.Index | None = None

print("RAGコンポーネント用のグローバル変数が初期化されました。")

RAGコンポーネント用のグローバル変数が初期化されました。


## RAGコンポーネント初期化関数定義と実行

In [8]:
# --- RAGコンポーネント初期化 ---

def initialize_rag_components(pdf_path: Path, embed_model_name: str, chunk_size: int, chunk_overlap: int):
    global CHUNKS_GLOBAL, EMBEDDER_GLOBAL, INDEX_GLOBAL # グローバル変数を変更することを明示

    print(f"RAGコンポーネントの初期化を開始します...")
    print(f"PDFファイルの読み込みとチャンク分割: {pdf_path}")
    raw_text = extract_text(pdf_path) # セル3で定義した関数

    if not raw_text.strip():
        print("警告: PDFからテキストが抽出できませんでした。PDFの内容、または extract_text 関数を確認してください。")
        CHUNKS_GLOBAL = []
    else:
        CHUNKS_GLOBAL = chunk_text(raw_text, chunk_size, chunk_overlap) # セル3で定義した関数

    if not CHUNKS_GLOBAL:
        print("警告: チャンクが生成されませんでした。PDFの内容、またはチャンキングのパラメータを確認してください。RAGは機能しません。")
        INDEX_GLOBAL = None
        EMBEDDER_GLOBAL = None
        return

    print(f"{len(CHUNKS_GLOBAL)}個のチャンクをエンベディングモデル ({embed_model_name}) でエンベディングします...")
    try:
        EMBEDDER_GLOBAL = SentenceTransformer(embed_model_name)
        embeddings = EMBEDDER_GLOBAL.encode(
            CHUNKS_GLOBAL, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True
        )

        if embeddings.ndim == 1: # チャンクが1つの場合、次元を調整
            embeddings = np.expand_dims(embeddings, axis=0)

        if embeddings.shape[0] == 0: # エンベディング結果が空の場合
             print("警告: エンベディング結果が空です。エンベディングモデルまたはチャンク内容を確認してください。")
             INDEX_GLOBAL = None
             return

        print(f"エンベディング完了。ベクトル次元: {embeddings.shape[1]}")
        INDEX_GLOBAL = faiss.IndexFlatIP(embeddings.shape[1]) # 内積 (Inner Product) で類似度を計算
        INDEX_GLOBAL.add(embeddings.astype(np.float32)) # FAISSはfloat32を要求
        print("FAISSインデックスの構築が完了しました。")
    except Exception as e:
        print(f"エンベディングモデルのロードまたはエンベディング処理中にエラーが発生しました: {e}")
        EMBEDDER_GLOBAL = None
        INDEX_GLOBAL = None
        return

    print("RAGコンポーネントの初期化が正常に完了しました。")

# --- RAGコンポーネント初期化の実行 ---
# ARGS_PARAM はセル2で定義されている想定です
print("initialize_rag_components を呼び出します...")
initialize_rag_components(
    ARGS_PARAM.pdf_path,
    ARGS_PARAM.embed_model,
    ARGS_PARAM.chunk_size,
    ARGS_PARAM.chunk_overlap
)
print("initialize_rag_components の実行が完了しました。")

initialize_rag_components を呼び出します...
RAGコンポーネントの初期化を開始します...
PDFファイルの読み込みとチャンク分割: 令和6年4月1日施行.pdf
PDFファイルを開いています: 令和6年4月1日施行.pdf
合計ページ数: 58
  10ページ目までのテキストを抽出完了...
  20ページ目までのテキストを抽出完了...
  30ページ目までのテキストを抽出完了...
  40ページ目までのテキストを抽出完了...
  50ページ目までのテキストを抽出完了...
  58ページ目までのテキストを抽出完了...
PDFからのテキスト抽出が完了しました。
テキストをチャンク分割中... (全長: 165488, サイズ: 500, オーバーラップ: 200)
チャンク分割完了。生成されたチャンク数: 551
551個のチャンクをエンベディングモデル (intfloat/multilingual-e5-large) でエンベディングします...


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

エンベディング完了。ベクトル次元: 1024
FAISSインデックスの構築が完了しました。
RAGコンポーネントの初期化が正常に完了しました。
initialize_rag_components の実行が完了しました。


## 検索関数定義

In [9]:
# --- 検索関数 ---

def search_chunks(query: str, top_k: int) -> List[Tuple[str, float, int]]:
    """指定されたクエリに最も関連性の高いチャンクを検索し、上位top_k件を返します。

    戻り値: (チャンクテキスト, 類似度スコア, チャンクインデックス) のタプルのリスト
    """
    global CHUNKS_GLOBAL, EMBEDDER_GLOBAL, INDEX_GLOBAL # グローバル変数を参照

    if INDEX_GLOBAL is None or EMBEDDER_GLOBAL is None or not CHUNKS_GLOBAL:
        print("警告: RAGコンポーネントが初期化されていないため、検索を実行できません。")
        return []

    # print(f"検索クエリ: \"{query[:50]}...\" (上位{top_k}件)")
    try:
        q_vec = EMBEDDER_GLOBAL.encode([query], convert_to_numpy=True, normalize_embeddings=True)
        scores, idxs = INDEX_GLOBAL.search(q_vec.astype(np.float32), min(top_k, len(CHUNKS_GLOBAL)))

        results = []
        for j, i in enumerate(idxs[0]):
            if i < 0 or i >= len(CHUNKS_GLOBAL): # 不正なインデックスを除外
                # print(f"  警告: 不正なインデックス {i} が検出されました。スキップします。")
                continue
            results.append((CHUNKS_GLOBAL[i], float(scores[0][j]), i))

        # print(f"  検索結果 {len(results)} 件:")
        # for text, score, idx in results:
        #     print(f"    スコア: {score:.4f}, インデックス: {idx}, テキスト: \"{text[:30]}...\"")
        return results
    except Exception as e:
        print(f"チャンク検索中にエラーが発生しました: {e}")
        return []

print("検索関数 (search_chunks) が定義されました。")

検索関数 (search_chunks) が定義されました。


## Gemini API呼び出し関数定義

In [10]:
# --- Gemini API 呼び出し関数 ---

def call_gemini(system_instruction: str, user_content: str, model_name: str, temperature: float, max_tokens: int) -> str:
    """Geminiモデルに指示とユーザーコンテンツを送信し、テキスト応答を取得します。"""
    # print(f"Gemini API ({model_name}) を呼び出します...")
    # print(f"  システム指示: \"{system_instruction[:50]}...\"")
    # print(f"  ユーザーコンテンツ: \"{user_content[:50]}...\"")

    try:
        model = genai.GenerativeModel(
            model_name=model_name,
            system_instruction=system_instruction
        )
        response = model.generate_content(
            user_content,
            generation_config=genai.GenerationConfig(
                temperature=temperature,
                max_output_tokens=max_tokens
            ),
            # safety_settings=[ # 必要に応じてセーフティセッティングを調整
            #     {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            #     {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            #     {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            #     {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            # ]
        )
        # print(f"  Gemini APIからの応答取得成功。")
        return response.text.strip()
    except Exception as e:
        print(f"Gemini API呼び出し中にエラーが発生しました ({model_name}): {e}")
        # エラーの詳細情報を確認 (例: response.prompt_feedback)
        # if hasattr(e, 'response') and hasattr(e.response, 'prompt_feedback'):
        #     print(f"  Prompt Feedback: {e.response.prompt_feedback}")
        # elif hasattr(e, 'message'): # 一部の google.api_core.exceptions で見られる
        # print(f"  Error message: {e.message}")

        return "回答を生成できませんでした (APIエラー)。詳細はログを確認してください。"

print("Gemini API呼び出し関数 (call_gemini) が定義されました。")

Gemini API呼び出し関数 (call_gemini) が定義されました。


## 回答生成関数定義 (ベースライン & RAG)

In [11]:
# --- 回答生成関数 (ベースライン & RAG) ---
# ARGS_PARAM はセル2で定義されている想定です (型ヒントで使用)
# セル2で定義した ArgsNamespace クラスをここで参照できるようにしておいてください。

def answer_baseline(question: str, args: ArgsNamespace) -> str:
    """ベースラインLLM（RAGなし）で質問に回答します。"""
    print(f"  ベースライン回答を生成中 (モデル: {args.gemini_model})...")
    system_prompt = "あなたは日本の特許法に詳しい法律専門家です。質問に対して、あなたが持つ知識のみに基づいて回答してください。"
    user_prompt = (
        f"質問: {question}\n\n"
        "回答を日本語で端的に述べてください。可能であれば、関連する条番号など具体的に示してください。"
    )
    return call_gemini(system_prompt, user_prompt, args.gemini_model, args.temperature, args.max_output_tokens)

def answer_rag(question: str, args: ArgsNamespace) -> Tuple[str, List[Dict[str, Any]]]:
    """RAGを使用して質問に回答し、使用したコンテキスト情報も返します。"""
    print(f"  RAG回答を生成中 (モデル: {args.gemini_model}, Top-K: {args.top_k})...")
    retrieved_contexts_for_log: List[Dict[str, Any]] = []

    # 検索関数 (search_chunks) はセル6で定義されている想定
    ctx_chunks_data = search_chunks(question, args.top_k)

    if not ctx_chunks_data:
        print("    RAG: 関連コンテキストが見つかりませんでした。")
        retrieved_contexts_for_log.append({"text": "No context found during search.", "score": 0.0, "index": -1, "note": "Search returned empty."})
        # コンテキストがない場合のプロンプト。LLMに知識だけで答えさせるか、不明と答えさせるか。
        # ここでは、コンテキストがないことを明示的にLLMに伝えます。
        context_for_prompt = "関連性の高い参照情報は見つかりませんでした。あなたの知識に基づいて回答を試みてください。"
    else:
        print(f"    RAG: {len(ctx_chunks_data)}件の関連コンテキストを検索しました。")
        context_parts = []
        for i, (text, score, chunk_idx) in enumerate(ctx_chunks_data):
            context_parts.append(
                f"関連情報 {i+1} (類似度スコア: {score:.4f}, 内部インデックス: {chunk_idx}):\n{text}"
            )
            retrieved_contexts_for_log.append({"text": text, "score": float(score), "index": int(chunk_idx)})
        context_for_prompt = "\n---\n".join(context_parts)


    system_prompt = "あなたは優秀な法律専門家です。提供された以下の「コンテキスト情報」を最優先の根拠として、質問に回答してください。"
    user_prompt = (
        "以下のコンテキスト情報を参考に、質問に答えてください。\n"
        "### コンテキスト情報\n" + context_for_prompt + "\n\n"
        "### 質問\n" + question + "\n\n"
        "回答は日本語で、根拠となる条文番号を可能な限り (特許法XX条) の形で明示してください。\n"
        "コンテキスト情報に適切な回答が含まれていない場合、またはコンテキスト情報だけでは判断できない場合は、その旨を正直に述べてください。\n"
        "コンテキスト情報と矛盾する回答はしないでください。"
    )

    response_text = call_gemini(system_prompt, user_prompt, args.gemini_model, args.temperature, args.max_output_tokens)
    return response_text, retrieved_contexts_for_log

print("回答生成関数 (answer_baseline, answer_rag) が定義されました。")

回答生成関数 (answer_baseline, answer_rag) が定義されました。


## 評価関数定義 (Exact Match)

In [12]:
# --- 評価関数 (Exact Match) ---

def exact_match(pred: str, gold: str) -> bool:
    """単純な文字列一致（空白と大文字・小文字を無視）で正誤を判定します。"""
    if not pred or not gold: # 片方または両方が空の場合は不一致とする
        return False
    pred_processed = "".join(pred.split()).lower()
    gold_processed = "".join(gold.split()).lower()
    return gold_processed in pred_processed # goldがpredに含まれていればTrue (より柔軟な場合)
    # return pred_processed == gold_processed # 完全一致の場合

print("評価関数 (exact_match) が定義されました。")

評価関数 (exact_match) が定義されました。


## LLMによる評価関数定義

In [13]:
def evaluate_by_llm(question: str, generated_answer: str, gold_answer: str, args: ArgsNamespace) -> Dict[str, Any]:
    """(開発中) 生成された回答をLLMを使用して評価します。"""
    print(f"    LLMによる評価を実行中 (モデル: {args.gemini_model})...") # 評価専用モデルを使うことも検討
    system_prompt = "あなたは回答評価アシスタントです。提供された質問、模範解答、および生成された回答に基づいて、生成された回答の品質を評価してください。"
    user_prompt = f"""以下の情報に基づいて、生成された回答を評価してください。

### 質問
{question}

### 模範解答
{gold_answer}

### 生成された回答
{generated_answer}

### 評価項目
1. 正確性 (Accuracy): 生成された回答は事実として正しいか、模範解答と整合性があるか？ (0-5点で評価、5が最高)
2. 完全性 (Completeness): 生成された回答は質問に対して必要な情報を網羅しているか？ (0-5点で評価、5が最高)
3. 関連性 (Relevance): 生成された回答は質問に適切に対応しているか？ (0-5点で評価、5が最高)
4. 明瞭性 (Clarity): 生成された回答は明確で理解しやすいか？ (0-5点で評価、5が最高)

上記の各評価項目について、0から5点の整数で評価し、それぞれの点数と簡単な根拠をJSON形式で出力してください。
例: {{"accuracy": 5, "accuracy_reason": "模範解答と完全に一致しており、事実として正しい。", "completeness": 4, "completeness_reason": "主要な情報は網羅されているが、一部詳細が不足。", ...}}
"""
    # 評価にはより低コスト・高速なモデルや、評価専用にチューニングされたモデルを使うことも考えられます。
    # ここでは args.gemini_model を流用しますが、必要に応じて変更してください。
    # 温度は低め (例: 0.1) が安定した評価結果を得やすいでしょう。
    eval_model_name = args.gemini_model # または "gemini-1.5-flash-latest" など
    eval_temperature = 0.1
    eval_max_tokens = 512 # 評価結果のJSONが収まる程度

    raw_eval_response = call_gemini(system_prompt, user_prompt, eval_model_name, eval_temperature, eval_max_tokens)

    # LLMからの応答が期待通りJSON形式であることを確認し、パースする
    try:
        # Geminiの応答は時々 ```json ... ``` のようにマークダウンで囲まれることがあるため、それを取り除く処理
        if raw_eval_response.startswith("```json"):
            raw_eval_response = raw_eval_response.strip("```json\n").strip("`\n")
        elif raw_eval_response.startswith("```"):
             raw_eval_response = raw_eval_response.strip("```\n").strip("`\n")

        parsed_eval = json.loads(raw_eval_response)
    except json.JSONDecodeError:
        print(f"    LLM評価結果のJSONパースに失敗しました。生応答: {raw_eval_response}")
        parsed_eval = {"error": "failed to parse LLM evaluation response", "raw_response": raw_eval_response}
    except Exception as e:
        print(f"    LLM評価中に予期せぬエラー: {e}")
        parsed_eval = {"error": f"unexpected error during LLM evaluation: {str(e)}", "raw_response": raw_eval_response}
    return parsed_eval

print("LLMによる評価関数 (evaluate_by_llm) が定義されました。")

LLMによる評価関数 (evaluate_by_llm) が定義されました。


## 評価実行関数定義 (run_eval)

In [14]:
# --- 評価実行関数 ---
# ARGS_PARAM はセル2で定義されている想定です

def run_eval(questions_data: List[Dict[str, str]], args: ArgsNamespace) -> Dict[str, Any]:
    """提供された質問リストに対してベースラインとRAGの評価を実行します。"""
    print(f"評価プロセスを開始します。合計 {len(questions_data)} 件の質問を処理します。")
    evaluation_results: Dict[str, Any] = {}

    for q_idx, q_data in enumerate(questions_data):
        q_id = q_data.get("id", f"q_unknown_{q_idx+1}")
        question_text = q_data.get("text")
        gold_answer = q_data.get("answer")
        q_notes = q_data.get("notes", "") # 質問に関するメモ (HOMEWORK.md用)

        if not question_text or not gold_answer:
            print(f"  警告: 質問ID {q_id} のテキストまたは模範解答が不足しています。スキップします。")
            continue

        print(f"\n処理中の質問 {q_idx + 1}/{len(questions_data)}: ID \"{q_id}\"")
        print(f"  質問文: \"{question_text[:70]}...\"")

        # ベースライン回答の生成
        baseline_ans = answer_baseline(question_text, args)
        print(f"    ベースライン回答: \"{baseline_ans[:70]}...\"")

        # RAG回答の生成
        rag_ans, retrieved_contexts = answer_rag(question_text, args)
        print(f"    RAG回答: \"{rag_ans[:70]}...\"")

        # 手動評価用のテンプレート (結果JSONに含め、後で手動で記入する)
        manual_eval_template = {
            "accuracy": None, "completeness": None, "relevance": None, "clarity": None, # 評価項目例
            "accuracy_reason": "", "completeness_reason": "", "relevance_reason": "", "clarity_reason": "",
            "other_comments": ""
        }

        # (オプション) LLMによる自動評価をここで呼び出す場合
        # if 'evaluate_by_llm' in globals(): # 関数が定義されていれば
        #     print(f"  LLMによる自動評価を実行中...")
        #     llm_eval_baseline = evaluate_by_llm(question_text, baseline_ans, gold_answer, args)
        #     llm_eval_rag = evaluate_by_llm(question_text, rag_ans, gold_answer, args)
        # else:
        #     llm_eval_baseline = {"status": "not_executed"}
        #     llm_eval_rag = {"status": "not_executed"}


        current_q_results = {
            "question_text": question_text,
            "gold_answer": gold_answer,
            "question_notes": q_notes,
            "baseline_answer": baseline_ans,
            "rag_answer": rag_ans,
            "baseline_exact_match": exact_match(baseline_ans, gold_answer), # セル9で定義
            "rag_exact_match": exact_match(rag_ans, gold_answer),           # セル9で定義
            "retrieved_contexts_for_rag": retrieved_contexts,
            "manual_evaluation_baseline": manual_eval_template.copy(),
            "manual_evaluation_rag": manual_eval_template.copy(),
            # "llm_auto_evaluation_baseline": llm_eval_baseline, # LLM評価を使う場合
            # "llm_auto_evaluation_rag": llm_eval_rag,           # LLM評価を使う場合
        }
        evaluation_results[q_id] = current_q_results

        print(
            f"  評価結果 (Exact Match): ベースライン={'✅' if current_q_results['baseline_exact_match'] else '❌'} | "
            f"RAG={'✅' if current_q_results['rag_exact_match'] else '❌'}"
        )

    print("\n全ての質問の評価処理が完了しました。")
    return evaluation_results

print("評価実行関数 (run_eval) が定義されました。")

評価実行関数 (run_eval) が定義されました。


## 質問データのロードとメイン処理の実行

In [15]:
# --- メイン処理の実行 ---
# ARGS_PARAM はセル2で定義されている想定です
# 依存する関数 (initialize_rag_components, run_eval など) はこれより前のセルで定義・実行済みであること。

print("メイン処理を開始します...")

# 1. 質問データのロード
questions_to_run_param: List[Dict[str, str]] # 型ヒント
# DEFAULT_QUESTIONS_DATA_PARAM はセル2で定義されている想定
if ARGS_PARAM.questions_path.exists():
    print(f"質問ファイルをロード中: {ARGS_PARAM.questions_path}")
    try:
        with open(ARGS_PARAM.questions_path, "r", encoding="utf-8") as f:
            questions_to_run_param = json.load(f)
        print(f"  {len(questions_to_run_param)} 件の質問をロードしました。")
    except Exception as e:
        print(f"  質問ファイルのロードに失敗しました: {e}. デフォルトの質問データを使用します。")
        questions_to_run_param = DEFAULT_QUESTIONS_DATA_PARAM
else:
    print(f"警告: 質問ファイルが見つかりません ({ARGS_PARAM.questions_path})。デフォルトの質問データを使用します。")
    questions_to_run_param = DEFAULT_QUESTIONS_DATA_PARAM

if not questions_to_run_param:
    print("エラー: 実行する質問データがありません。処理を中断します。")
    all_results_param = {} # 結果を空にする
else:
    # 2. RAGコンポーネントが利用可能か確認
    # (セル5で initialize_rag_components が実行されていれば、INDEX_GLOBAL の状態で判断可能)
    if INDEX_GLOBAL is None and EMBEDDER_GLOBAL is None : # RAGなしモード、または初期化失敗
        print("警告: RAGコンポーネントが初期化されていないか、失敗しています。RAG評価は限定的になるか、実行されません。")
        # 必要ならここでRAGなしの評価のみを行うなどの分岐処理

    # 3. 評価の実行
    # run_eval 関数はセル11で定義されている想定
    print("\n評価を開始します...")
    all_results_param = run_eval(questions_to_run_param, ARGS_PARAM)
    print("評価が完了しました。")

    # 4. 結果の保存
    if all_results_param: # 結果が空でない場合のみ保存
        # parameters 内のPathオブジェクトを文字列に変換する
        serializable_params = {}
        for k, v in vars(ARGS_PARAM).items():
            if isinstance(v, Path):
                serializable_params[k] = str(v)
            else:
                # DEFAULT_QUESTIONS_DATA_PARAM のような大きなリストは保存しないようにする (任意)
                if k == "default_questions_data": # このキー名はセル2の定義に合わせる
                    serializable_params[k] = f"<{len(v)} default questions>"
                else:
                    serializable_params[k] = v

        final_output_param = {
            "run_id": ARGS_PARAM.run_id,
            "timestamp": datetime.datetime.now().isoformat(),
            "parameters": serializable_params,
            "evaluation_results": all_results_param
        }

        # 出力ディレクトリの作成
        ARGS_PARAM.output_dir.mkdir(parents=True, exist_ok=True)
        output_filename_param = ARGS_PARAM.output_dir / f"rag_eval_results_{ARGS_PARAM.run_id}.json"

        print(f"\n詳細な結果をファイルに保存します: {output_filename_param}")
        try:
            with open(output_filename_param, "w", encoding="utf-8") as fp:
                json.dump(final_output_param, fp, ensure_ascii=False, indent=2)
            print("結果の保存が完了しました。")
        except Exception as e:
            print(f"結果のファイル保存中にエラーが発生しました: {e}")
    else:
        print("\n保存する評価結果がありません。")

print("\n全てのメイン処理が完了しました。")
print("次のステップ:")
print("1. 生成されたJSON結果ファイル (`results_notebook` フォルダ内など) を確認してください。")
print("2. JSONファイル内の `manual_evaluation_baseline` と `manual_evaluation_rag` セクションに手動で評価を記入してください。")
print("3. これらの結果と考察を `HOMEWORK.md` にまとめてください。")

メイン処理を開始します...
警告: 質問ファイルが見つかりません (questions.json)。デフォルトの質問データを使用します。

評価を開始します...
評価プロセスを開始します。合計 5 件の質問を処理します。

処理中の質問 1/5: ID "q_default_1"
  質問文: "発明の新規性喪失の例外（特許法30条）の適用を受けるには、証明書を特許出願の日から何日以内に提出する必要がありますか？..."
  ベースライン回答を生成中 (モデル: gemini-2.5-pro-exp-03-25)...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 873.58ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 483.61ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 558.02ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1189.35ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2099.74ms


    ベースライン回答: "特許出願の日から30日以内です。

これは、特許法第30条第3項及び特許法施行規則第25条第1項に規定されています。..."
  RAG回答を生成中 (モデル: gemini-2.5-pro-exp-03-25, Top-K: 5)...
    RAG: 5件の関連コンテキストを検索しました。
    RAG回答: "発明の新規性喪失の例外（特許法30条）の適用を受けるための証明書は、特許出願の日から三十日以内に特許庁長官に提出する必要があります (特許法..."
  評価結果 (Exact Match): ベースライン=✅ | RAG=❌

処理中の質問 2/5: ID "q_default_2"
  質問文: "仮専用実施権は、どのような場合に移転が認められますか？具体的に列挙してください。..."
  ベースライン回答を生成中 (モデル: gemini-2.5-pro-exp-03-25)...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 380.90ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 380.99ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 481.61ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 405.77ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 582.26ms


    ベースライン回答: "..."
  RAG回答を生成中 (モデル: gemini-2.5-pro-exp-03-25, Top-K: 5)...
    RAG: 5件の関連コンテキストを検索しました。


...

    ベースライン回答: "特許権の存続期間延長登録の出願ができる場合の「政令で定めるもの」に該当しない場合は、主に以下の2つのケースが考えられます。

1.  **特..."
  RAG回答を生成中 (モデル: gemini-2.5-pro-exp-03-25, Top-K: 5)...
    RAG: 5件の関連コンテキストを検索しました。


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 33877.95ms


    RAG回答: "..."
  評価結果 (Exact Match): ベースライン=❌ | RAG=❌

処理中の質問 5/5: ID "q_default_5"
  質問文: "令和6年4月1日施行の改正特許法において、意匠権の存続期間満了後のいわゆる「グレースピリオド」に関する新たな規定は導入されましたか？その内容..."
  ベースライン回答を生成中 (モデル: gemini-2.5-pro-exp-03-25)...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 734.38ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 7309.33ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro-exp-03-25:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 558.76ms


    ベースライン回答: "..."
  RAG回答を生成中 (モデル: gemini-2.5-pro-exp-03-25, Top-K: 5)...
    RAG: 5件の関連コンテキストを検索しました。
    RAG回答: "コンテキスト情報には、令和6年4月1日施行の改正特許法における意匠権の存続期間満了後のいわゆる「グレースピリオド」に関する新たな規定について..."
  評価結果 (Exact Match): ベースライン=❌ | RAG=❌

全ての質問の評価処理が完了しました。
評価が完了しました。

詳細な結果をファイルに保存します: results_notebook/rag_eval_results_20250512_070831_notebook.json
結果の保存が完了しました。

全てのメイン処理が完了しました。
次のステップ:
1. 生成されたJSON結果ファイル (`results_notebook` フォルダ内など) を確認してください。
2. JSONファイル内の `manual_evaluation_baseline` と `manual_evaluation_rag` セクションに手動で評価を記入してください。
3. これらの結果と考察を `HOMEWORK.md` にまとめてください。
